In [ ]:
import pandas as pd
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Define number of records
num_records = 100

Generating Dummy Data:

Uses Faker to generate random data for 100 employees.
Creates a pandas DataFrame from the generated data.
Saves the DataFrame to a CSV file (hr_records.csv).

In [ ]:
# Generate dummy data
data = {
    'EmployeeID': [i for i in range(1, num_records + 1)],
    'FirstName': [fake.first_name() for _ in range(num_records)],
    'LastName': [fake.last_name() for _ in range(num_records)],
    'Age': [random.randint(20, 60) for _ in range(num_records)],
    'Gender': [random.choice(['Male', 'Female']) for _ in range(num_records)],
    'Department': [random.choice(['HR', 'IT', 'Finance', 'Marketing', 'Sales']) for _ in range(num_records)],
    'Salary': [round(random.uniform(30000, 120000), 2) for _ in range(num_records)],
    'HireDate': [fake.date_between(start_date='-10y', end_date='today') for _ in range(num_records)],
    'Email': [fake.email() for _ in range(num_records)]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv('open_ai_chatbot_talk_with_sqldatabase/data/hr_records.csv', index=False)

print("Dummy HR data generated and saved to hr_records.csv")

Creating SQLite Database:

Defines a SQLite database filename (hr_database.db).
Uses SQLAlchemy to create a SQLite engine.
Defines the Employee table using SQLAlchemy ORM.
Creates the table in the SQLite database.
Converts the DataFrame to a list of dictionaries.
Inserts the records into the employees table.
Commits the transaction and closes the session.


In [ ]:
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, MetaData, Table, inspect, text
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, declarative_base

In [ ]:
# Define the SQLite database file name
database_filename = 'open_ai_chatbot_talk_with_sqldatabase/data/hr_database.db'

# Create SQLite engine
engine = create_engine(f'sqlite:///{database_filename}', echo=True)

# Define a base class for the ORM
Base = declarative_base()

# Define the Employee table as an ORM class
class Employee(Base):
    __tablename__ = 'employees'
    EmployeeID = Column(Integer, primary_key=True, autoincrement=True)
    FirstName = Column(String)
    LastName = Column(String)
    Age = Column(Integer)
    Gender = Column(String)
    Department = Column(String)
    Salary = Column(Float)
    HireDate = Column(Date)
    Email = Column(String)

# Create the table in the database
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Convert DataFrame to list of dictionaries and remove EmployeeID
records = df.drop(columns=['EmployeeID']).to_dict(orient='records')

# Insert records into the database
for record in records:
    employee = Employee(**record)
    session.add(employee)

# Commit the transaction
session.commit()

print(f"Database '{database_filename}' created and records inserted.")


In [ ]:
# Function to show table names
def show_table_names():
    try:
        inspector = inspect(engine)
        tables = inspector.get_table_names()
        print("Tables in the database:")
        for table in tables:
            print(table)
    except Exception as e:
        print(f"An error occurred while fetching table names: {e}")
    
show_table_names()

In [ ]:
# Test the connection by reading from the database
def test_connection_and_read():
    try:
        # Test the connection
        conn = engine.connect()
        print("Successfully connected to the database.")

        # Read the contents of the employees table
        result = conn.execute(text("SELECT * FROM employees"))

        # Print the contents of the table
        for row in result:
            print(row)

        # Close the connection
        conn.close()
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the test function
test_connection_and_read()

# Close the session
session.close()